In [2]:
import pandas as pd
from consts import PATH, TEST_PATH, TRAIN_PATH
from train_test_data_split import TrainTestSplitter

In [2]:
TrainTestSplitter(left_part_days=180).save_splited_rfm()

22:35:29 Save splitted rfm
22:35:29 Getting splited alive partners rfm
22:35:29 Alive partners calculation
22:35:40 Creating RFM
22:36:40 Data spliting


In [3]:
data = pd.read_parquet(TRAIN_PATH)
data

,monetary_value,first_buy,last_buy,count,alive,frequency,recency,T
partner,,,,,,,,
0,5.698337e-10,2021-08-27,2022-01-27,8,True,7,153,545
1,1.712412e-10,2021-08-23,2022-01-27,17,True,16,157,549
2,2.016771e-09,2021-09-25,2021-09-25,1,True,0,0,516
4,1.748972e-09,2021-10-19,2022-02-04,5,False,4,108,492
9,1.858965e-10,2021-08-22,2022-01-10,7,True,6,141,550
...,...,...,...,...,...,...,...,...
1229908,1.445898e-12,2022-02-01,2022-02-01,1,True,0,0,387
1229910,6.024121e-14,2022-02-01,2022-02-01,1,True,0,0,387
1229914,4.461669e-13,2021-10-25,2022-02-01,2,False,1,99,486


In [4]:
(data['first_buy'].max() - data['first_buy'].min()).days

179

In [9]:
def calc_destribution(data: dict) -> dict:
    data_sum = sum(data.values())
    distribution = {}
    for key, value in data.items():
        distribution[key] = f"{round(value / data_sum, 3) * 100:.1f}%"
    return distribution

In [11]:
last_data_date = data['first_buy'].max()
data['pred_alive'] = data['last_buy'].apply(lambda x: (last_data_date - x).days < 59)
data['corr_pred'] = data['pred_alive'] == data['alive']
calc_destribution(data['corr_pred'].value_counts().to_dict())

{True: '79.4%', False: '20.6%'}

In [12]:
from btyd.utils import calculate_alive_path, expected_cumulative_transactions
import pandas as pd
import numpy as np
import pickle

from consts import MODEL_PATH_180_DAYS, TRAIN_RAW_PATH, TEST_RAW_PATH, TEST_PATH, TRAIN_PATH

data = pd.read_parquet(TRAIN_RAW_PATH)
rfm = pd.read_parquet(TRAIN_PATH)
model = pickle.load(open(MODEL_PATH_180_DAYS, "rb"))

partner = 1229915
t = rfm.loc[partner]["T"]
transactions = data[data["partner"]==partner]
datetime_col = "rep_date"
freq="D"
start_date=None
ax=None

customer_history = transactions[[datetime_col]].copy()
customer_history.index = pd.DatetimeIndex(customer_history[datetime_col])

# Add transactions column
customer_history["transactions"] = 1
customer_history = customer_history.resample(freq).sum()

# plot alive_path
path = calculate_alive_path(model, transactions, datetime_col, t, freq)
path_dates = pd.date_range(start=min(transactions[datetime_col]), periods=len(path), freq=freq)

In [20]:
data["rep_date"].max()

datetime.date(2021, 12, 15)

In [16]:
path_dates

DatetimeIndex(['2021-09-12', '2021-09-13', '2021-09-14', '2021-09-15',
               '2021-09-16', '2021-09-17', '2021-09-18', '2021-09-19',
               '2021-09-20', '2021-09-21',
               ...
               '2023-02-14', '2023-02-15', '2023-02-16', '2023-02-17',
               '2023-02-18', '2023-02-19', '2023-02-20', '2023-02-21',
               '2023-02-22', '2023-02-23'],
              dtype='datetime64[ns]', length=530, freq='D')

In [13]:
path

0                         [1.0]
1                         [1.0]
2                         [1.0]
3                         [1.0]
4                         [1.0]
                 ...           
525     [0.0003189140797317343]
526     [0.0003156739978894152]
527    [0.00031247277575673164]
528    [0.00030930987636267727]
529     [0.0003061847711307631]
Length: 530, dtype: object